<img style="float: right; margin: 5px 5px 20px 20px;" src="https://upload.wikimedia.org/wikipedia/commons/d/db/Logo_ITESO_normal.jpg" 
width="100px" height="75px"/>

# 003 Deep Learning


### Microstructures and trading systems


In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import optuna
import ta

In [29]:
data = pd.read_csv("aapl_5m_train.csv").dropna()
data.head()

,Unnamed: 0,Timestamp,Gmtoffset,Datetime,Open,High,Low,Close,Volume
0,0,1609770600,0,2021-01-04 14:30:00,133.570007,133.611602,132.389999,132.809997,6624663.0
1,1,1609770900,0,2021-01-04 14:35:00,132.750000,132.750000,131.809997,131.889999,2541553.0
2,2,1609771200,0,2021-01-04 14:40:00,131.500000,132.339996,131.500000,132.059997,2492415.0
3,3,1609771500,0,2021-01-04 14:45:00,132.000000,132.250000,131.899993,132.250000,1859131.0
4,4,1609771800,0,2021-01-04 14:50:00,132.000000,132.018096,131.520004,131.589996,1780105.0


In [31]:
rsi = ta.momentum.RSIIndicator(data.Close, window=20)
bb = ta.volatility.BollingerBands(data.Close, window =15, window_dev = 2)
macd = ta.trend.MACD(data.Close, window_slow=26, window_fast=12, window_sign=9)

In [33]:
dataset = data.copy()
dataset['RSI'] = rsi.rsi()
dataset['BB_MAVG'] = bb.bollinger_mavg()
dataset['BB_LOWER'] = bb.bollinger_lband()
dataset['BB_UPPER'] = bb.bollinger_hband()
dataset['MACD'] = macd.macd()
dataset['MACD_SIGNAL'] = macd.macd_signal()


# Signals
dataset['RSI_BUY'] = dataset['RSI'] < 25
dataset['RSI_SELL'] = dataset['RSI'] > 75

dataset['BB_BUY'] = bb.bollinger_lband_indicator().astype(bool)
dataset['BB_SELL'] = bb.bollinger_hband_indicator().astype(bool)

dataset['MACD_BUY'] = dataset['MACD'] > dataset['MACD_SIGNAL']
dataset['MACD_SELL'] = dataset['MACD'] < dataset['MACD_SIGNAL']

dataset = dataset.dropna()
dataset.head()

,Unnamed: 0,Timestamp,Gmtoffset,Datetime,Open,High,Low,Close,Volume,RSI,...,BB_LOWER,BB_UPPER,MACD,MACD_SIGNAL,RSI_BUY,RSI_SELL,BB_BUY,BB_SELL,MACD_BUY,MACD_SELL
33,33,1609780500,0,2021-01-04 17:15:00,127.410003,128.029998,126.760002,128.005004,2891333.0,26.630231,...,127.604115,130.914780,-0.872535,-0.728380,False,False,False,False,False,True
34,34,1609780800,0,2021-01-04 17:20:00,128.000793,128.399993,127.750000,128.289993,1956101.0,30.634171,...,127.466667,130.842001,-0.856193,-0.753943,False,False,False,False,False,True
35,35,1609781100,0,2021-01-04 17:25:00,128.289993,128.320007,127.889999,128.250000,1331407.0,30.389196,...,127.366537,130.696799,-0.836823,-0.770519,False,False,False,False,False,True
36,36,1609781400,0,2021-01-04 17:30:00,128.246902,128.310104,127.910003,128.119995,1314757.0,29.579796,...,127.315651,130.453684,-0.822481,-0.780911,False,False,False,False,False,True
37,37,1609781700,0,2021-01-04 17:35:00,128.125000,128.210006,127.889999,127.955001,982464.0,28.563455,...,127.235757,130.261391,-0.815033,-0.787736,False,False,False,False,False,True


In [44]:
capital = 1_000_000
com = 0.125 / 100

portfolio_value = [capital]

stop_loss = 0.15
take_profit = 0.10
n_shares = 1000

wins = 0
losses = 0

active_long_positions = None
active_short_positions = None

for i, row in dataset.iterrows():
    # Close Long Positions
    if active_long_positions:
        #Closed By Stop Loss
        if row.Close < active_long_positions['stop_loss']:
            pnl = row.Close * n_shares * (1 - com)
            capital += pnl
            active_long_positions = None
        
        # Closed By Take Profit
        elif row.Close > active_long_positions[take_profit]:
            pnl = row.Close * n_shares * (1 - com)
            capital += pnl
            active_long_positions = None

        # Close Short Positiones
        if active_short_positions:
            if row.Close > active_short_positions['stop_loss']:
                pnl = (active_short_positions['opened_at'] - row.Close) * n_shares * (1 - com)
                capital += pnl
                active_short_positions = None
            elif row.Close < active_short_positions['take_profit']:
                pnl = (active_short_positions['opened_at'] - row.Close) * n_shares * (1 - com)
                capital += pnl
                active_short_positions = None


        # Open Long Positiones
        if row.RSI_BUY and active_long_positions is None:
            cost = row.Close * n_shares * (1 + com)
            if capital > cost:
                capital -= cost
                active_long_positions = {
                    'datetime': row.Datetime,
                    'opened_at': row.Close,
                    'take_profit': row.Close * (1 + take_profit),
                    'stop_loss': row.Close * (1 - stop_loss)
                }

        # Open Short Positions
         if (row.RSI_SELL or row.BB_SELL or row.MACD_SELL) and active_short_positions is None:
            gain = row.Close * n_shares * (1 - com)
            capital += gain
            active_short_positions = {
                'datetime': row.Datetime,
                'opened_at': row.Close,
                'take_profit': row.Close * (1 - take_profit),
                'stop_loss': row.Close * (1 + stop_loss)
            }

        
        ## Calculate Portfolio Value
        # Calculate Long Positions Value
        long_value = (row.Close * n_shares) if active_long_positions else 0
    
        # Calculate Short Positions Value
        short_value = (active_short_positions['opened_at'] - row.Close) * n_shares if active_short_positions else 0
    
        # Add Portfolio Value
        portfolio_value.append(capital + long_value + short_value)


IndentationError: unindent does not match any outer indentation level (<string>, line 56)

In [46]:
plt.figure(figsize=(12, 6))
plt.plot(portfolio_value, label= 'Portfolio Value')
plt.legend()
plt.show()

NameError: name 'portfolio_value' is not defined

<Figure size 1200x600 with 0 Axes>

In [49]:
#RSI
def objective_func(trial, data):
    rsi_window = trial.suggest_int('rsi_window', 10, 100)
    rsi_lower =  trial.suggest_int('rsi_lower', 5, 35)
    rsi_upper =  trial.suggest_int('rsi_upper', 65, 95)

    stop_loss = trial.suggest_float('stop_loss', 0.01, 0.2)
    take_profit = trial.suggest_float('take_profit', 0.01, 0.2)
    n_shares = trial.suggest_categorical('n_shares', [100, 500, 800, 1000, 1200])
    
    rsi = ta.momentum.RSIIndicator(data.Close, window=20)
    bb = ta.volatility.BollingerBands(data.Close, window =15, window_dev = 2)
    macd = ta.trend.MACD(data.Close, window_slow=26, window_fast=12, window_sign=9)
    
    dataset = data.copy()
    dataset['RSI'] = rsi.rsi()
    dataset['BB_MAVG'] = bb.bollinger_mavg()
    dataset['BB_LOWER'] = bb.bollinger_lband()
    dataset['BB_UPPER'] = bb.bollinger_hband()
    dataset['MACD'] = macd.macd()
    dataset['MACD_SIGNAL'] = macd.macd_signal()
    
    
    # Signals
    dataset['RSI_BUY'] = dataset['RSI'] < 25
    dataset['RSI_SELL'] = dataset['RSI'] > 75
    
    dataset['BB_BUY'] = bb.bollinger_lband_indicator().astype(bool)
    dataset['BB_SELL'] = bb.bollinger_hband_indicator().astype(bool)
    
    dataset['MACD_BUY'] = dataset['MACD'] > dataset['MACD_SIGNAL']
    dataset['MACD_SELL'] = dataset['MACD'] < dataset['MACD_SIGNAL']
    
    dataset = dataset.dropna()
    
    capital = 1_000_000
    com = 0.125 / 100
    
    portfolio_value = [capital]
    
    stop_loss = 0.15
    take_profit = 0.10
    n_shares = 1000
    
    wins = 0
    losses = 0
    
    active_long_positions = None
    active_short_positions = None
    
    for i, row in dataset.iterrows():
        # Close Long Positions
        if active_long_positions:
            #Closed By Stop Loss
            if row.Close < active_long_positions['stop_loss']:
                pnl = row.Close * n_shares * (1 - com)
                capital += pnl
                active_long_positions = None
            
            # Closed By Take Profit
            elif row.Close > active_long_positions[take_profit]:
                pnl = row.Close * n_shares * (1 - com)
                capital += pnl
                active_long_positions = None
    
            # Close Short Positiones
            if active_short_positions:
                if row.Close > active_short_positions['stop_loss']:
                    pnl = (active_short_positions['opened_at'] - row.Close) * n_shares * (1 - com)
                    capital += pnl
                    active_short_positions = None
                elif row.Close < active_short_positions['take_profit']:
                    pnl = (active_short_positions['opened_at'] - row.Close) * n_shares * (1 - com)
                    capital += pnl
                    active_short_positions = None
    
    
            # Open Long Positiones
            if row.RSI_BUY and active_long_positions is None:
                cost = row.Close * n_shares * (1 + com)
                if capital > cost:
                    capital -= cost
                    active_long_positions = {
                        'datetime': row.Datetime,
                        'opened_at': row.Close,
                        'take_profit': row.Close * (1 + take_profit),
                        'stop_loss': row.Close * (1 - stop_loss)
                    }
    
            # Open Short Positions
             if (row.RSI_SELL or row.BB_SELL or row.MACD_SELL) and active_short_positions is None:
                gain = row.Close * n_shares * (1 - com)
                capital += gain
                active_short_positions = {
                    'datetime': row.Datetime,
                    'opened_at': row.Close,
                    'take_profit': row.Close * (1 - take_profit),
                    'stop_loss': row.Close * (1 + stop_loss)
                }

            portfolio_value.append(capital)
            
        return (np.mean(portfolio_value) / np.std(portfolio_value))  # Sharpe Ratio
            


IndentationError: unindent does not match any outer indentation level (<string>, line 91)

In [51]:
study = optuna.create_study(direction = 'maximize')
study.optimize(lambda x: objective_func(x, data), n_trials = 50)

[I 2025-03-27 14:58:36,295] A new study created in memory with name: no-name-867b3bc6-7cd9-4146-b903-47bd9cbb61e8
[W 2025-03-27 14:58:36,297] Trial 0 failed with parameters: {} because of the following error: NameError("name 'objective_func' is not defined").
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/var/folders/46/p3w40sm501lfx91hwfw5sfmh0000gp/T/ipykernel_46908/328805031.py", line 2, in <lambda>
    study.optimize(lambda x: objective_func(x, data), n_trials = 50)
                             ^^^^^^^^^^^^^^
NameError: name 'objective_func' is not defined
[W 2025-03-27 14:58:36,317] Trial 0 failed with value None.


NameError: name 'objective_func' is not defined

In [53]:
print("Best Parameters:", study.best_params)

ValueError: No trials are completed yet.

In [55]:
# Optimización
study = optuna.create_study(direction="maximize")
study.optimize(lambda x: objective_fun(x, data), n_trials=10)


[I 2025-03-27 14:58:38,555] A new study created in memory with name: no-name-20ff5bda-42b6-44f7-b282-86703ddbcba1
[W 2025-03-27 14:58:38,556] Trial 0 failed with parameters: {} because of the following error: NameError("name 'objective_fun' is not defined").
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/var/folders/46/p3w40sm501lfx91hwfw5sfmh0000gp/T/ipykernel_46908/1026619266.py", line 3, in <lambda>
    study.optimize(lambda x: objective_fun(x, data), n_trials=10)
                             ^^^^^^^^^^^^^
NameError: name 'objective_fun' is not defined
[W 2025-03-27 14:58:38,564] Trial 0 failed with value None.


NameError: name 'objective_fun' is not defined